In [44]:
import requests
from bs4 import BeautifulSoup
import csv

In [45]:
myFile = open('corolla.csv', 'w')

In [46]:
url = 'https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [47]:
totalPages = soup.select('.last.next a')[0]
totalPages = int(totalPages['href'].split('=')[-1])

In [48]:
def page(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.select('table.table-engine-detail > tbody > tr td')
    modelDate = int(table[0].text.rstrip())
    kmRun = table[1].text.rstrip()
    energy = table[2].text.rstrip()
    engineType = table[3].text.rstrip()
    carDetails = soup.select('#scroll_car_detail li')
    city,color,assembly,engineCapacity = "","","",""
    if(carDetails[0].text.rstrip() == "Registered City"):
        city = carDetails[1].text.rstrip()
    if(carDetails[2].text.rstrip() == "Color"):
        color = carDetails[3].text.rstrip()
    if(carDetails[4].text.rstrip() == "Assembly"):
        assembly = carDetails[5].text.rstrip()
    if(carDetails[6].text.rstrip() == "Engine Capacity"):
        engineCapacity = carDetails[7].text.rstrip()

    carFeatures = soup.select('.car-feature-list li')
    features = []
    for feature in carFeatures:
        features.append(feature.text.rstrip().strip())
    imagesDiv = soup.select('#myCarousel ul li')
    images = [""] * 5
    for i in range(min(5,len(imagesDiv))):
        image = imagesDiv[i].select('img')
        if(image):
            images[i]= image[0]['data-original']
            
    breadcrumb = soup.select('.breadcrumb li')
    make = breadcrumb[2].text.rstrip().strip()
    model = breadcrumb[3].text.rstrip().strip().split()[-1]
    version = " ".join(breadcrumb[5].text.rstrip().split()[2:-1])
    return make,model,version,images,modelDate,kmRun,energy,engineType,city,color,assembly,engineCapacity,features

In [49]:
writer = csv.writer(myFile)
with myFile:
    count = 0
    writer.writerows([["ID","name", "Make","Model","Version","Price","Link", "Model Date", "Km Run", "Energy", "Engine Type", "City", "Color", "Assembly", "EngineCapacity", "image1","image2","image3","image4","image5","Features"]])
    for i in range(1,totalPages+1):
        
        pageurl = url + "?page=" + str(i)
        r = requests.get(pageurl)
        soup = BeautifulSoup(r.content, 'html.parser')
        carDivs = soup.select('li .search-list')
        

        for i in range(len(carDivs)):
            try:
                count += 1
                info = carDivs[i].select('.col-md-9.grid-style')[0]
                price = info.select('.price-details')[0].text.rstrip()
                linkName = info.select('a')[0]
                link = "https://www.pakwheels.com"+linkName['href'].rstrip()
                name = linkName.text.rstrip()
                details = info.select('.row .grid-date ul')
                city = details[0].text.rstrip().strip()
                make,model,version,images,modelDate,kmRun,energy,engineType,city,color,assembly,engineCapacity,features = page(link)
                writer.writerows([[count,name,make,model,version,price,link,modelDate,kmRun,energy,engineType,city,color,assembly,engineCapacity,images[0],images[1],images[2],images[3],images[4],features]])       
            except Exception as e:
                print(e)
#             break
#         break
        print(pageurl)

https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=1
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=2
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=3
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=4
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=5
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=6
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=7
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=8
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=9
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=10
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=11
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=12
https://www.pakwheels.com/used-cars/search/-/mk_toyota/md_corolla/?page=13
https://www.pakwheels.com/used-car

KeyboardInterrupt: 